# 第一章 1.3 Jieba分词

## 1.3.1 基于N-gram模型的中文分词

### N-gram模型原理及其在分词中的应用

**N-gram模型原理**  
N-gram是一种基于统计的语言模型，假设当前词的出现概率仅依赖于前N-1个词。数学表达为：\[ P(w_i|w_1,w_2,...,w_{i-1}) \approx P(w_i|w_{i-n+1},...,w_{i-1}) \]

**在分词中的应用**  
1. 二元模型（Bigram）最常用，计算相邻词的共现概率：\[ P(W) = \prod_{i=2}^n P(w_i|w_{i-1}) \]  
2. 通过统计语料库中词序列频率，选择概率最大的分词组合。例如："研究生命"可切分为"研究/生命"（P=0.8）或"研究生/命"（P=0.2），模型会选择前者。

### 1.3.2基于隐马尔可夫模型（HMM）的中文分词

**HMM建模过程**  
定义五元组 \( \lambda = (S, O, A, B, \pi) \)：状态集S：{B（词首）, M（词中）, E（词尾）, S（单字词）} ，观测序列O：待分词的字符序列，状态转移矩阵A：B→M→E的概率，发射矩阵B：状态生成特定字符的概率（如"的"在S状态概率高），初始概率π：首字符各状态的分布

**分词实现方法**  
 Viterbi算法求解最优状态序列  

### 1.3.3 Jieba分词原理与流程

**核心算法**  
 *前缀词典*：加载预定义的词频词典（如"北京大学"：频次=1000），构建Trie树实现高效前缀匹配，时间复杂度O(k)（k为词长）
 *动态规划*（DP）：对未登录词，用DP求最大概率路径：\[ \max \prod_{i=1}^n P(w_i) \]，其中 \( P(w_i) \) 通过语料统计获得（如"中国"的log概率=-8.3）

**分词流程**  
 *初始化*：加载词典（35万条词，默认文件`dict.txt`），*切分阶段*：用Trie树匹配所有可能成词，对歧义句（如"结婚的和尚未结婚的"）生成DAG图，*消歧阶段*：对DAG执行Viterbi算法（HMM模型处理未登录词），联合词典概率和HMM结果选择最优切分，*结果调整*：处理特殊规则（如英文、数字合并）

In [8]:
def forward_max_matching(vocab, sentence):
    """
    基于正向最大匹配算法 (FMM) 的中文分词函数

    参数：
    vocab (list): 词典，包含了所有可能的词汇
    sentence (str): 需要分词的句子

    返回：
    list: 分词结果的列表
    """
    fmmresult = []  # 存储分词结果的列表
    max_len = max([len(item) for item in vocab])  # 获取词典中最长词的长度
    start = 0  # 分词的起始位置

    # 开始遍历句子，直到处理完整个句子
    while start != len(sentence):
        index = start + max_len  # 尝试匹配最大长度的词
        if index > len(sentence):  # 如果索引超出句子长度，修正为句子末尾
            index = len(sentence)

        # 从当前起始位置尝试从最大长度开始逐步缩小词的长度进行匹配
        while index > start:
            current_substr = sentence[start:index]  # 截取当前子串
            # 如果子串在词典中，或者子串长度为1，则认为是一个有效词
            if current_substr in vocab or len(current_substr) == 1:
                fmmresult.append(current_substr)  # 将有效词加入结果列表
                start = index  # 更新起始位置，跳过已处理的部分
                break  # 找到一个有效词后跳出内层循环继续处理下一个子串
            index -= 1  # 如果没有匹配到有效词，缩短子串长度再试

    return fmmresult  # 返回最终的分词结果

In [9]:
def reverse_directional_max_matching(vocab, sentence):
    """
    Reverse Maximum Matching (RMM) 分词算法。
    从句子的末尾开始，尝试从词典中匹配最长的词直到句子被分割。

    Args:
    vocab (list): 词典，包含了所有可能的词汇
    sentence (str): 需要分词的句子

    返回：
    list: 分词后的结果，按顺序返回分词列表
    """
    rmmresult = []  # 存储分词结果
    max_len = max([len(item) for item in vocab])  # 获取词典中最大词的长度
    start = len(sentence)  # 从句子的末尾开始

    while start != 0:  # 直到处理完整个句子
        index = start - max_len  # 尝试从当前位置往前推最大长度的子串
        if index < 0:
            index = 0  # 防止下标越界，调整为从0开始

        while index < start:  # 向前查找直到找到匹配的词
            current_substr = sentence[index:start]  # 截取当前子串

            # 如果当前子串在词典中，或当前子串长度为1（即单个字符）
            if current_substr in vocab or len(current_substr) == 1:
                rmmresult.insert(0, current_substr)  # 匹配成功，插入到结果列表的开头
                start = index  # 更新起始位置，继续向前匹配
                break  # 找到一个词后跳出内层循环
            index += 1  # 如果当前子串没有匹配，向前移动一个字符继续尝试

    return rmmresult  # 返回最终的分词结果

In [10]:
def bi_directional_matching(vocab, sentence):
    # 获取正向和反向最大匹配的分词结果
    res1 = forward_max_matching(vocab, sentence)
    res2 = reverse_directional_max_matching(vocab, sentence)

    len_res1, len_res2 = len(res1), len(res2)  # 保存长度

    # 如果两个结果的长度相同
    if len_res1 == len_res2:
        # 如果两个结果相同，直接返回
        if res1 == res2:
            return res1
        else:
            # 统计每个结果中长度为1的词的数量
            res1_sn = sum(1 for i in res1 if len(i) == 1)
            res2_sn = sum(1 for i in res2 if len(i) == 1)
            # 返回包含较少单字符词的结果
            return res1 if res1_sn < res2_sn else res2
    else:
        # 返回词数较少的结果
        return res1 if len_res1 < len_res2 else res2

In [11]:
vocab = ['我们', '今天', '在', '野生动物园', '玩']
sentence = '我们是今天在野生动物园玩了'

In [12]:
fmm_result = forward_max_matching(vocab, sentence)
rmm_result = reverse_directional_max_matching(vocab, sentence)
print(fmm_result)
print(rmm_result)

['我们', '是', '今天', '在', '野生动物园', '玩', '了']
['我们', '是', '今天', '在', '野生动物园', '玩', '了']


In [13]:
bm_result = bi_directional_matching(vocab, sentence)
print(bm_result)

['我们', '是', '今天', '在', '野生动物园', '玩', '了']
